In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline

In [3]:
data_iterator = pd.read_csv("meter_values_all.csv", chunksize=100000, index_col = 'Unnamed: 0')

chunk_list = []  

# Each chunk is in dataframe format
for data_chunk in data_iterator:  
    new_chunk = data_chunk#.astype({'StationId': np.int8, 'UnitId': np.int8}).drop(['StationTransactionId', 'MeterUnit', 'ValueStr'],axis=1)
    chunk_list.append(new_chunk)
meter_values = pd.concat(chunk_list)

In [4]:
meter_values.FieldType.value_counts()

POWER                                   11496429
CURRENT                                 10427455
VOLTAGE                                  9755920
VOLTAGE_AFTER_CONTACTOR                  6672800
TEMPERATURE                              6659881
ERROR_STATE                              4906372
ENERGY_REGISTER                          3427969
TARGET_CURRENT                           2712706
SLAC_AAG                                 2566382
ADDITIONAL_TEMPERATURE                   2315489
ENERGY_ACTIVE_IMPORT_REGISTER_FROM_0     1236887
TARGET_CURRENT_LIMIT                      724264
PING                                      665206
MOBILE_SIGNAL_TYPE                        650108
BATTERY_LEVEL                             151606
MOBILE_SIGNAL_STRENGTH                    131488
HLC_STATE                                 117420
CURRENT_MAX                               104565
VOLTAGE_INPUT                              92054
POWER_MAX                                  66999
TARGET_VOLTAGE      

In [3]:
meter_values

,StationId,UnitId,Time,Value,FieldType,LocationType
50583020,13,1,2022-01-01 00:00:00,73.0,ERROR_STATE,CABLE
56438898,14,1,2022-01-01 00:00:00,12.0,TEMPERATURE,CABLE
57651280,15,0,2022-01-01 00:00:00,16.0,ADDITIONAL_TEMPERATURE,Y_CPU
50583021,13,1,2022-01-01 00:00:00,76.0,ERROR_STATE,CABLE
34087765,38,1,2022-01-01 00:00:00,691.0,VOLTAGE,Y_E_METER_INPUT
...,...,...,...,...,...,...
25228109,5,1,2022-01-31 23:59:59,7.0,TEMPERATURE,CABLE
38479646,9,1,2022-01-31 23:59:59,12.0,TEMPERATURE,CABLE
34087764,37,16,2022-01-31 23:59:59,64.0,ERROR_STATE,Y_CONVERTER_MODULE
62124289,15,1,2022-01-31 23:59:59,7.0,TEMPERATURE,CABLE


In [3]:
# meter_values = meter_values.sort_values(['Time', 'StationId'], ascending=True)
# meter_values_7 = meter_values.iloc[0:12464684]
# meter_values_7.to_csv('meter_values_7.csv')

In [13]:
meter_values.set_index('StationId')

,UnitId,Time,Value,FieldType,LocationType
StationId,,,,,
13,1,2022-01-01 00:00:00,73.0,ERROR_STATE,CABLE
14,1,2022-01-01 00:00:00,12.0,TEMPERATURE,CABLE
15,0,2022-01-01 00:00:00,16.0,ADDITIONAL_TEMPERATURE,Y_CPU
13,1,2022-01-01 00:00:00,76.0,ERROR_STATE,CABLE
38,1,2022-01-01 00:00:00,691.0,VOLTAGE,Y_E_METER_INPUT
...,...,...,...,...,...
5,1,2022-01-31 23:59:59,7.0,TEMPERATURE,CABLE
9,1,2022-01-31 23:59:59,12.0,TEMPERATURE,CABLE
37,16,2022-01-31 23:59:59,64.0,ERROR_STATE,Y_CONVERTER_MODULE


In [37]:
meter_values_count = meter_values.pivot_table(index=['UnitId'], 
                        columns='FieldType', 
                        values='Value', 
                        aggfunc='count',
                        fill_value=0).reset_index()

In [6]:
list_of_time = []
def timing(time):
    for x in time:
        x = x.split(sep= ' +')
        x = x[0].split(sep = '.')
        list_of_time.append(x[0]) 
timing(meter_values.Time)

In [13]:
meter_values.dtypes

StationId          int8
UnitId             int8
Time             object
Value           float64
FieldType        object
LocationType     object
dtype: object

In [16]:
# new_meter_values = (meter_values['Time'] >= '2022-01-01 00:05:11.835 +0000') & (meter_values['Time'] <= '2022-01-07 23:59:59.835 +0000')
# new_meter_values = meter_values.loc[new_meter_values]
# #new_meter_values.to_csv('new_meter_values.csv')

In [17]:
# station_state = pd.read_csv('station_state_january.csv').drop(['StationTransactionId','TimestampEventRegistered','BoardErrorCode'], axis=1)
# station_state.head(5)


,StationId,Time,UnitId,StationUnitType,StationUnitState,Comment
0,1,2022-01-01 00:05:11.835 +0000,0,SUB_HIGH_CPU_USAGE,OK,Average load (20%) in last 10 min
1,1,2022-01-01 00:06:24.347 +0000,2,LG_PU_CONV,CRITICAL,HasNoAnswerFromBoard
2,1,2022-01-01 00:06:27.447 +0000,2,LG_PU_CONV,OK,NaN
3,1,2022-01-01 00:30:18.083 +0000,0,SUB_HIGH_CPU_USAGE,OK,Average load (19%) in last 10 min
4,1,2022-01-01 00:35:59.539 +0000,0,SUB_HIGH_CPU_USAGE,OK,Average load (20%) in last 10 min


In [45]:
# new_station_state = (station_state['Time'] >= '2022-01-01 00:05:11.835 +0000') & (station_state['Time'] <= '2022-01-07 23:59:59.835 +0000')
# new_station_state = station_state.loc[new_station_state]
# #new_station_state.to_csv('new_station_state.csv')

In [19]:
# timing(station_state.Time)
# station_state['Time'] = list_of_time

In [28]:
# station_state = station_state.astype({'StationId': np.int8, 'UnitId': np.int8})

In [32]:
station_state.shape

(10590282, 6)

In [4]:
# meter_values = pd.read_csv('meter_values_all.csv')
station_state = pd.read_csv('station_state.csv', index_col='Unnamed: 0')

In [6]:
station_state

,StationId,Time,UnitId,StationUnitType,StationUnitState,Comment
0,1,2022-01-01 00:05:11,0,SUB_HIGH_CPU_USAGE,OK,Average load (20%) in last 10 min
1,1,2022-01-01 00:06:24,2,LG_PU_CONV,CRITICAL,HasNoAnswerFromBoard
2,1,2022-01-01 00:06:27,2,LG_PU_CONV,OK,NaN
3,1,2022-01-01 00:30:18,0,SUB_HIGH_CPU_USAGE,OK,Average load (19%) in last 10 min
4,1,2022-01-01 00:35:59,0,SUB_HIGH_CPU_USAGE,OK,Average load (20%) in last 10 min
...,...,...,...,...,...,...
2692926,38,2022-01-31 23:56:34,2,LG_TEMPERATURE,OK,"CPU, 35℃"
2692927,38,2022-01-31 23:56:40,1,LG_TEMPERATURE,OK,"CPU, 21℃"
2692928,38,2022-01-31 23:56:40,2,LG_TEMPERATURE,OK,"CPU, 21℃"
2692929,38,2022-01-31 23:57:05,0,SUB_HIGH_CPU_USAGE,OK,Average load (33%) in last 10 min


In [8]:
station_state_count = station_state.pivot_table(index='StationId', 
                        columns='StationUnitState', 
                        values='UnitId', 
                        aggfunc='count',
                        fill_value=0).reset_index()

In [10]:
station_state_count.sort_values(['CRITICAL'], ascending=False)

StationUnitState,StationId,CRITICAL,ERROR,INFO,OK,WARNING
12,13,60315,175808,88,18305,137
1,2,21715,25058,748,17615,0
13,14,5568,63,1,29851,177
18,37,2459,194,40,131579,9
17,36,1615,139,5,152837,17
19,38,1598,249,116,146413,7
9,10,1047,30,0,26355,0
0,1,902,788,10,26101,9
2,3,676,42703,22,97906,1541
6,7,607,278,1,23778,374


In [23]:
station_state.StationUnitType.unique()

array(['SUB_HIGH_CPU_USAGE', 'LG_PU_CONV', 'LG_CPU', 'LG_PU_OUT',
       'LG_CHILLER', 'LG_E_METER_OUT', 'LG_NETWORK_MODEM', 'LG_PLC_MODEM',
       'LG_TEMPERATURE', 'APP_OCPP', 'APP_V2G', 'APP_MONITOR',
       'APP_LOCAL_MANAGER', 'SUB_CAN_BUS', 'SUB_HIGH_FS_USAGE',
       'SUB_HIGH_MEMORY_USAGE', 'SUB_VPN', 'SUB_POSTGRES',
       'LG_E_METER_IN', 'LG_CAMERA', 'LG_CONTACTORS_INTERNAL'],
      dtype=object)

In [5]:
meter_values.FieldType.unique()

array(['ERROR_STATE', 'TEMPERATURE', 'ADDITIONAL_TEMPERATURE', 'VOLTAGE',
       'CURRENT', 'POWER', 'ENERGY_REGISTER', 'CURRENT_MAX',
       'VOLTAGE_MAX', 'POWER_MAX', 'VOLTAGE_AFTER_CONTACTOR',
       'VOLTAGE_INPUT', 'VEHICLE_CONNECT', 'CHARGING_ALLOW',
       'CONTACTOR_LOCKED', 'ISOLATION_STATE', 'BOARD_READY',
       'CP_LINE_STATE', 'FAN_ACTIVE', 'DOOR_OPEN', 'FIRE_EMERGENCY',
       'HLC_STATE', 'EV_ERROR_STATE', 'CHARGING_GOING', 'BATTERY_LEVEL',
       'TARGET_CURRENT', 'TARGET_VOLTAGE', 'TARGET_CURRENT_LIMIT',
       'TARGET_VOLTAGE_LIMIT', 'TARGET_POWER_LIMIT',
       'TARGET_CHARGING_ALLOW', 'EVCC_ID', 'PING',
       'MOBILE_SIGNAL_STRENGTH', 'MOBILE_SIGNAL_TYPE', 'CONNECTOR_LOCKED',
       'AUTHENTICATION', 'CHILLER_STATE', 'CHILLER_WATER_RUN', 'RESET',
       'SLAC_AAG', 'ENERGY_ACTIVE_IMPORT_REGISTER_FROM_0',
       'STOP_TRANSACTION', 'PWM_ENABLED', 'POWER_DIRECTION_MODE',
       'ELECTRICITY_TYPE', 'PROTOCOL_TYPE', 'FORCE_MODE_ENABLE',
       'FORCE_MODE_VALUE', 'CUR

In [5]:
station_state = station_state.query('StationId == [13, 14, 15, 38,  3,  8,  1, 12, 37, 16,  9,  2, 10,  6,  5, 11, 17, 4,  7, 36]')

In [42]:
meter_values.StationId.unique()

array([13, 14, 15, 38,  3,  8,  1, 12, 37, 16,  9,  2, 10,  6,  5, 11, 17,
        4,  7, 36], dtype=int64)

In [59]:
station_state.StationId.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       36, 37, 38], dtype=int64)

In [7]:
meter_values['full_info'] = meter_values['LocationType'].astype(str) + '_' + meter_values['FieldType'].astype(str) 

In [28]:
abiba = pd.DataFrame(meter_values.full_info.value_counts()).reset_index()

In [29]:
abiba

,index,full_info
0,CABLE_VOLTAGE,8075555
1,CABLE_CURRENT,7989304
2,CABLE_POWER,7985650
3,CABLE_VOLTAGE_AFTER_CONTACTOR,6672799
4,CABLE_TEMPERATURE,4841982
...,...,...
89,OUTLET_POWER_ACTIVE_IMPORT,2
90,OUTLET_POWER_FACTOR,2
91,OUTLET_SOC,2
92,OUTLET_ENERGY_ACTIVE_IMPORT_REGISTER_FROM_0,2


In [4]:
meter_values.full_info.unique()

array(['CABLE_ERROR_STATE', 'CABLE_TEMPERATURE',
       'Y_CPU_ADDITIONAL_TEMPERATURE', 'Y_E_METER_INPUT_VOLTAGE',
       'Y_CONVERTER_MODULE_ERROR_STATE', 'CABLE_CURRENT', 'Y_CPU_CURRENT',
       'Y_E_METER_CURRENT', 'Y_E_METER_INPUT_CURRENT', 'CABLE_VOLTAGE',
       'Y_CPU_VOLTAGE', 'Y_E_METER_VOLTAGE', 'CABLE_POWER', 'Y_CPU_POWER',
       'Y_E_METER_POWER', 'Y_E_METER_INPUT_POWER',
       'Y_E_METER_ENERGY_REGISTER', 'Y_E_METER_INPUT_ENERGY_REGISTER',
       'Y_CPU_CURRENT_MAX', 'Y_LOGIC_BOARD_CURRENT_MAX',
       'Y_CPU_VOLTAGE_MAX', 'Y_LOGIC_BOARD_VOLTAGE_MAX',
       'Y_CPU_POWER_MAX', 'Y_LOGIC_BOARD_POWER_MAX',
       'CABLE_VOLTAGE_AFTER_CONTACTOR', 'Y_CPU_VOLTAGE_INPUT',
       'CABLE_VEHICLE_CONNECT', 'Y_HLC_CHARGING_ALLOW',
       'CABLE_CONTACTOR_LOCKED', 'Y_CPU_CONTACTOR_LOCKED',
       'CABLE_ISOLATION_STATE', 'Y_CPU_TEMPERATURE',
       'Y_LOGIC_BOARD_TEMPERATURE', 'Y_CPU_BOARD_READY',
       'Y_HLC_BOARD_READY', 'CABLE_CP_LINE_STATE', 'Y_CPU_FAN_ACTIVE',
       'Y_HLC_D

In [55]:
# meter_values[(meter_values.UnitId == 1) & (meter_values.StationId == 13) & \
#     (meter_values['Time'] >= '2022-01-31 21:00:01') & (meter_values['Time'] <= '2022-01-31 22:50:05')]


In [56]:
# station_state[(station_state.UnitId == 2) & (station_state.StationId == 13) &\
#     (station_state['Time'] >= '2022-01-31 21:00:50') & (station_state['Time'] <= '2022-01-31 22:50:10')]

In [45]:
station_state_count = station_state.pivot_table(index='StationUnitType', 
                        columns='StationUnitState', 
                        values='UnitId', 
                        aggfunc='count',
                        fill_value=0).reset_index()

In [54]:
#тип подсистемы и самые частые 'critical'
station_state_count.sort_values(['CRITICAL'], ascending=False)

StationUnitState,StationUnitType,CRITICAL,ERROR,INFO,OK,WARNING
12,LG_PU_OUT,60976,176590,0,13126,0
8,LG_E_METER_OUT,22079,54782,0,59213,0
11,LG_PU_CONV,14602,14655,123,64584,9893
6,LG_CPU,653,57,0,10533,0
7,LG_E_METER_IN,500,3,0,2283,0
5,LG_CHILLER,222,379,0,5482,0
13,LG_TEMPERATURE,213,0,0,575758,0
14,SUB_CAN_BUS,42,4347,0,14721,0
9,LG_NETWORK_MODEM,33,0,0,6900,0
4,LG_CAMERA,3,0,0,14508,0


In [52]:
#тип подсистемы и меньше всего 'critical'
station_state_count.sort_values(['CRITICAL', ], ascending=True)

StationUnitState,StationUnitType,CRITICAL,ERROR,INFO,OK,WARNING
0,APP_LOCAL_MANAGER,0,193,0,11157,0
17,SUB_HIGH_MEMORY_USAGE,0,6,0,13368,38
16,SUB_HIGH_FS_USAGE,0,137,0,22025,0
15,SUB_HIGH_CPU_USAGE,0,1741,896,191945,0
10,LG_PLC_MODEM,0,0,0,7056,1
18,SUB_POSTGRES,0,24,0,6615,0
19,SUB_VPN,0,189,0,10690,0
3,APP_V2G,0,256,0,10939,0
2,APP_OCPP,0,1728,0,9231,0
1,APP_MONITOR,0,211,92,11141,0


In [49]:
#тип подсистемы и самые частые 'OK'
station_state_count.sort_values(['OK'], ascending=False)

StationUnitState,StationUnitType,CRITICAL,ERROR,INFO,OK,WARNING
13,LG_TEMPERATURE,213,0,0,575758,0
15,SUB_HIGH_CPU_USAGE,0,1741,896,191945,0
11,LG_PU_CONV,14602,14655,123,64584,9893
8,LG_E_METER_OUT,22079,54782,0,59213,0
16,SUB_HIGH_FS_USAGE,0,137,0,22025,0
14,SUB_CAN_BUS,42,4347,0,14721,0
4,LG_CAMERA,3,0,0,14508,0
17,SUB_HIGH_MEMORY_USAGE,0,6,0,13368,38
12,LG_PU_OUT,60976,176590,0,13126,0
0,APP_LOCAL_MANAGER,0,193,0,11157,0


In [11]:
meter_values_2 = meter_values[meter_values['StationId'] == 2].drop(['StationId', 'LocationType', 'FieldType', 'UnitId'], axis=1)

In [12]:
# meter_values_13.to_csv('meter_values_id_13.csv')
meter_values_13 = pd.read_csv('meter_values_id_13.csv', index_col='Unnamed: 0')

In [12]:
meter_values_2

,Time,Value,full_info
1202651,2022-01-01 00:00:02,44.0,Y_LOGIC_BOARD_TEMPERATURE
1202652,2022-01-01 00:00:02,25.0,Y_MODEM_PING
1202653,2022-01-01 00:00:02,3.0,Y_MODEM_MOBILE_SIGNAL_TYPE
1202654,2022-01-01 00:00:05,0.0,CABLE_CURRENT
1202655,2022-01-01 00:00:05,0.0,Y_E_METER_CURRENT
...,...,...,...
2433641,2022-01-28 14:04:36,0.0,Y_E_METER_POWER
2433642,2022-01-28 14:04:40,10.0,Y_E_METER_POWER
2433643,2022-01-28 14:04:41,0.0,Y_E_METER_POWER
2433644,2022-01-28 14:05:06,10.0,Y_E_METER_POWER


In [54]:
meter_values_13.Value.dtypes

dtype('float64')

In [14]:
meter_values_2['Time'] = pd.to_datetime(meter_values_2.Time)
meter_values_2['date'] = meter_values_2.Time.dt.date

In [15]:
station_state_2 = station_state[station_state['StationId'] == 2].drop(['StationId', 'UnitId'], axis=1)

In [25]:
station_state_13.StationUnitType.unique()

array(['LG_PU_OUT', 'LG_TEMPERATURE', 'SUB_HIGH_CPU_USAGE', 'LG_CPU',
       'LG_CHILLER', 'LG_E_METER_OUT', 'LG_PU_CONV', 'LG_NETWORK_MODEM',
       'LG_PLC_MODEM', 'APP_OCPP', 'APP_V2G', 'APP_MONITOR',
       'APP_LOCAL_MANAGER', 'SUB_CAN_BUS', 'SUB_HIGH_FS_USAGE',
       'SUB_HIGH_MEMORY_USAGE', 'SUB_VPN', 'SUB_POSTGRES'], dtype=object)

In [29]:
station_state_2 = station_state_2.query('(StationUnitType == ["LG_CHILLER", "LG_E_METER_OUT", "LG_PU_OUT", "LG_TEMPERATURE", "LG_PU_CONV", "LG_CPU", "LG_NETWORK_MODEM"]) & (StationUnitState == ["OK", "CRITICAL"])')
station_state_2.drop

In [30]:
station_state_13.StationUnitType.value_counts()

LG_PU_OUT           60147
LG_TEMPERATURE       2591
LG_PU_CONV           2182
LG_E_METER_OUT        574
LG_CPU                439
LG_CHILLER            301
LG_NETWORK_MODEM      287
Name: StationUnitType, dtype: int64

In [31]:
# station_state_13['Y'] = station_state_13['StationUnitType'].astype(str) + '_' + station_state_13['StationUnitState'].astype(str)

C:\Users\Kikikita\AppData\Local\Temp\ipykernel_14940\3210331738.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_state_13['Y'] = station_state_13['StationUnitType'].astype(str) + '_' + station_state_13['StationUnitState'].astype(str)


In [20]:
# station_state_2 = station_state_2.query('StationUnitState == ["CRITICAL", "OK"]')
station_state_2['y'] = station_list

In [25]:
station_state_2 = station_state_2.drop(['Comment'], axis=1)

In [26]:
station_state_2.to_csv('station_state_2.csv')

In [24]:
station_state_2.StationUnitState.value_counts()

CRITICAL    21715
OK          17615
Name: StationUnitState, dtype: int64

In [19]:
station_list = []
for i in station_state_2.StationUnitState:
    if i == 'CRITICAL':
        station_list.append(1)
    else:
        station_list.append(0)

In [18]:
station_state_2['Time'] = pd.to_datetime(station_state_2.Time)

In [23]:
station_state_2.query('(StationUnitType == ["LG_PU_OUT"]) & (StationUnitState == ["OK", "CRITICAL"])').StationUnitState.value_counts()

CRITICAL    751
OK           41
Name: StationUnitState, dtype: int64

In [27]:
time_meter_values_2 = meter_values_2.pivot_table(index='Time', 
                        columns='full_info', 
                        values='Value', 
                        fill_value=0).reset_index()

In [28]:
time_meter_values_2

full_info,Time,CABLE_CONNECTOR_LOCKED,CABLE_CONTACTOR_LOCKED,CABLE_CP_LINE_STATE,CABLE_CURRENT,CABLE_ERROR_STATE,CABLE_ISOLATION_STATE,CABLE_POWER,CABLE_TEMPERATURE,CABLE_VEHICLE_CONNECT,...,Y_MODEM_PING,Y_PU_BATTERY_LEVEL,Y_PU_CHARGING_GOING,Y_PU_HLC_STATE,Y_PU_POWER_DIRECTION_MODE,Y_PU_PROTOCOL_TYPE,Y_PU_PWM_ENABLED,Y_PU_TARGET_CURRENT_LIMIT,Y_PU_TARGET_POWER_LIMIT,Y_PU_TARGET_VOLTAGE_LIMIT
0,2022-01-01 00:00:02,0,0,0,0.0,0,0,0,0,0,...,25.0,0,0,0.0,0,0,0,0,0.0,0.0
1,2022-01-01 00:00:05,0,0,-116,0.0,88,0,0,0,0,...,25.0,0,0,0.0,0,0,0,0,0.0,0.0
2,2022-01-01 00:01:02,0,0,0,0.0,0,0,0,0,0,...,26.0,0,0,0.0,0,0,0,0,0.0,0.0
3,2022-01-01 00:02:02,0,0,0,0.0,0,0,0,0,0,...,32.0,0,0,0.0,0,0,0,0,0.0,0.0
4,2022-01-01 00:02:03,0,0,0,0.0,0,0,0,0,0,...,0.0,0,0,0.0,0,0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109178,2022-01-28 14:04:35,0,0,0,0.0,0,0,0,0,0,...,0.0,0,0,0.0,0,0,0,0,0.0,0.0
109179,2022-01-28 14:04:36,0,0,0,0.0,0,0,0,0,0,...,0.0,0,0,0.0,0,0,0,0,0.0,0.0
109180,2022-01-28 14:04:40,0,0,0,0.0,0,0,0,0,0,...,0.0,0,0,0.0,0,0,0,0,0.0,0.0
109181,2022-01-28 14:04:41,0,0,0,0.0,0,0,0,0,0,...,0.0,0,0,0.0,0,0,0,0,0.0,0.0


In [55]:
time_meter_values_2.dtypes

full_info
Time                                                  datetime64[ns]
CABLE_CONNECTOR_LOCKED                                         int64
CABLE_CONTACTOR_LOCKED                                       float64
CABLE_CP_LINE_STATE                                            int64
CABLE_CURRENT                                                float64
CABLE_ERROR_STATE                                            float64
CABLE_ISOLATION_STATE                                        float64
CABLE_POWER                                                  float64
CABLE_TEMPERATURE                                            float64
CABLE_VEHICLE_CONNECT                                        float64
CABLE_VOLTAGE                                                float64
CABLE_VOLTAGE_AFTER_CONTACTOR                                float64
EV_EVCC_ID                                                     int64
EV_TARGET_CHARGING_ALLOW                                     float64
EV_TARGET_CURRENT       

In [31]:
time_meter_values_2 = time_meter_values_2.drop(['Y_CONTACTOR_CONTACTOR_LOCKED',
'Y_PU_CONTACTOR_LOCKED',
'OUTLET_VOLTAGE',
'OUTLET_ENERGY_ACTIVE_IMPORT_REGISTER',
'OUTLET_POWER_ACTIVE_IMPORT',
'OUTLET_POWER_FACTOR',
'OUTLET_SOC',
'OUTLET_ENERGY_ACTIVE_IMPORT_REGISTER_FROM_0',
'OUTLET_CURRENT_IMPORT',
'Y_LOGIC_BOARD_RESET',
'Y_LOGIC_BOARD_STOP_TRANSACTION'], axis=1)

KeyError: "['Y_CONTACTOR_CONTACTOR_LOCKED', 'Y_PU_CONTACTOR_LOCKED', 'OUTLET_VOLTAGE', 'OUTLET_ENERGY_ACTIVE_IMPORT_REGISTER', 'OUTLET_POWER_ACTIVE_IMPORT', 'OUTLET_POWER_FACTOR', 'OUTLET_SOC', 'OUTLET_ENERGY_ACTIVE_IMPORT_REGISTER_FROM_0', 'OUTLET_CURRENT_IMPORT', 'Y_LOGIC_BOARD_RESET', 'Y_LOGIC_BOARD_STOP_TRANSACTION'] not found in axis"

In [29]:
time_meter_values_2

full_info,Time,CABLE_CONNECTOR_LOCKED,CABLE_CONTACTOR_LOCKED,CABLE_CP_LINE_STATE,CABLE_CURRENT,CABLE_ERROR_STATE,CABLE_ISOLATION_STATE,CABLE_POWER,CABLE_TEMPERATURE,CABLE_VEHICLE_CONNECT,...,Y_HLC_TARGET_VOLTAGE_LIMIT,Y_LOGIC_BOARD_CHILLER_WATER_RUN,Y_LOGIC_BOARD_CURRENT_MAX,Y_LOGIC_BOARD_ENERGY_ACTIVE_IMPORT_REGISTER_FROM_0,Y_LOGIC_BOARD_POWER_MAX,Y_LOGIC_BOARD_TEMPERATURE,Y_LOGIC_BOARD_VOLTAGE_MAX,Y_MODEM_MOBILE_SIGNAL_STRENGTH,Y_MODEM_MOBILE_SIGNAL_TYPE,Y_MODEM_PING
0,2022-01-01 00:00:00,0,0.0,0,0.0,74.5,0.0,0.0,0.0,0.0,...,0,0,0.0,0,0,0,0.0,0,0,0
1,2022-01-01 00:00:01,0,0.0,0,0.0,71.5,0.0,0.0,0.0,0.0,...,0,0,0.0,0,0,0,0.0,0,0,0
2,2022-01-01 00:00:02,0,0.0,0,0.0,73.0,0.0,0.0,0.0,0.0,...,0,0,0.0,0,0,36,0.0,0,3,42
3,2022-01-01 00:00:03,0,0.0,0,0.0,74.5,0.0,0.0,0.0,0.0,...,0,0,0.0,0,0,0,0.0,0,0,0
4,2022-01-01 00:00:04,0,0.0,0,0.0,71.5,0.0,0.0,0.0,0.0,...,0,0,0.0,0,0,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431146,2022-01-31 23:58:31,0,0.0,0,0.0,0.0,0.0,0.0,42.0,0.0,...,0,0,0.0,0,0,0,0.0,0,0,0
1431147,2022-01-31 23:58:37,0,0.0,0,0.0,0.0,0.0,0.0,43.0,0.0,...,0,0,0.0,0,0,0,0.0,0,0,0
1431148,2022-01-31 23:58:47,0,0.0,0,0.0,0.0,0.0,0.0,44.0,0.0,...,0,0,0.0,0,0,0,0.0,0,0,0
1431149,2022-01-31 23:58:51,0,0.0,0,0.0,0.0,0.0,0.0,43.0,0.0,...,0,0,0.0,0,0,0,0.0,0,0,0


In [58]:
merge_meter_state_13 = station_state_2.merge(time_meter_values_13, on='Time', how='outer')#.fillna(method ='pad')

In [30]:
pd.DataFrame(meter_values_2.full_info.value_counts()).tail(20)

,full_info
EV_TARGET_VOLTAGE,12
Y_LOGIC_BOARD_VOLTAGE_MAX,8
Y_CPU_FIRE_EMERGENCY,8
Y_LOGIC_BOARD_CURRENT_MAX,8
Y_CONTACTOR_FORCE_MODE_ENABLE,8
Y_LOGIC_BOARD_POWER_MAX,8
Y_E_METER_INPUT_POWER,8
Y_E_METER_INPUT_CURRENT,8
Y_E_METER_INPUT_VOLTAGE,8
Y_CPU_BOARD_READY,8


In [2]:
# merge_meter_state_13 = pd.read_csv('merge_meter_state_13.csv', index_col='Unnamed: 0')
# merge_meter_state_13 = merge_meter_state_13.drop(['Y_LOGIC_BOARD_RESET', 'OUTLET_CURRENT_IMPORT', 'OUTLET_ENERGY_ACTIVE_IMPORT_REGISTER_FROM_0', 'OUTLET_ENERGY_ACTIVE_IMPORT_REGISTER_FROM_0'])

In [50]:
merge_meter_state_13 = merge_meter_state_13.fillna(method ='pad')

In [64]:
merge_meter_state_13.drop(['StationUnitType', 'StationUnitState'], axis=1)


,Time,y,CABLE_CONNECTOR_LOCKED,CABLE_CONTACTOR_LOCKED,CABLE_CP_LINE_STATE,CABLE_CURRENT,CABLE_ERROR_STATE,CABLE_ISOLATION_STATE,CABLE_POWER,CABLE_TEMPERATURE,...,Y_HLC_TARGET_VOLTAGE_LIMIT,Y_LOGIC_BOARD_CHILLER_WATER_RUN,Y_LOGIC_BOARD_CURRENT_MAX,Y_LOGIC_BOARD_ENERGY_ACTIVE_IMPORT_REGISTER_FROM_0,Y_LOGIC_BOARD_POWER_MAX,Y_LOGIC_BOARD_TEMPERATURE,Y_LOGIC_BOARD_VOLTAGE_MAX,Y_MODEM_MOBILE_SIGNAL_STRENGTH,Y_MODEM_MOBILE_SIGNAL_TYPE,Y_MODEM_PING
0,2022-01-01 00:00:29,1,0.0,0.0,0.0,0.0,74.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-01-01 00:00:40,1,0.0,0.0,0.0,0.0,74.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022-01-01 00:01:24,1,0.0,0.0,0.0,0.0,74.5,0.0,0.0,42.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022-01-01 00:01:35,1,0.0,0.0,0.0,0.0,74.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022-01-01 00:01:46,1,0.0,0.0,0.0,0.0,74.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1442602,2022-01-31 23:58:31,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1442603,2022-01-31 23:58:37,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1442604,2022-01-31 23:58:47,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1442605,2022-01-31 23:58:51,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
merge_meter_state_13 = merge_meter_state_13.query('y == [0,1]').drop(['StationUnitType', 'StationUnitState'], axis=1)

In [66]:
merge_meter_state_13.y.value_counts()

1    60315
0    18305
Name: y, dtype: int64

In [72]:
merge_meter_state_13 = merge_meter_state_13.dropna()

In [73]:
merge_meter_state_13.to_csv('merge_meter_state_13.csv')

In [62]:
merge_meter_state_13['y'] = merge_meter_state_13.y.fillna(3).astype(int)

In [3]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [12]:
X = merge_meter_state_13.drop(['StationUnitType','StationUnitState', 'Y', 'Time'], axis=1)
y = merge_meter_state_13.Y

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=10)

In [14]:
clf = LinearSVC(C=1.27, random_state=10)
clf.fit(X_train, y_train)

c:\Users\Kikikita\anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(C=1.27, random_state=10)

In [26]:
y_pred = clf.predict(X_test).astype(str)

In [27]:
y_test = y_test.astype(str)

In [28]:
df = pd.DataFrame(y_test)
df['y_pred'] = y_pred

In [38]:
df

,Y,y_pred
595219,LG_PU_OUT_ERROR,LG_PU_OUT_ERROR
1199196,LG_PU_OUT_ERROR,LG_PU_OUT_ERROR
882371,LG_PU_OUT_ERROR,SUB_HIGH_CPU_USAGE_OK
939320,LG_PU_OUT_ERROR,LG_PU_OUT_ERROR
31498,LG_PU_OUT_ERROR,LG_PU_OUT_ERROR
...,...,...
111673,LG_PU_OUT_ERROR,LG_PU_OUT_ERROR
330917,LG_PU_OUT_CRITICAL,LG_PU_OUT_ERROR
637061,LG_PU_OUT_ERROR,LG_PU_OUT_ERROR
934188,LG_PU_OUT_ERROR,LG_PU_OUT_ERROR


In [18]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [39]:
recall_score(y_test, y_pred, average='macro')

0.0577654608581986

In [40]:
f1_score(y_test, y_pred, average='macro')

0.04983843550431377

In [179]:
merge_meter_state_13.StationUnitState.value_counts()

ERROR       997382
CRITICAL    338906
OK          111023
INFO           272
WARNING        157
Name: StationUnitState, dtype: int64

In [118]:
ok_13 = merge_meter_state_13[merge_meter_state_13.StationUnitState == 'OK']
ok_13

,Time,Value,full_info,StationUnitType,StationUnitState,Comment
812,2022-01-01 00:06:13,73.0,CABLE_ERROR_STATE,LG_TEMPERATURE,OK,"CPU, 31℃"
813,2022-01-01 00:06:13,31.0,Y_CPU_TEMPERATURE,LG_TEMPERATURE,OK,"CPU, 31℃"
949,2022-01-01 00:07:23,73.0,CABLE_ERROR_STATE,SUB_HIGH_CPU_USAGE,OK,Average load (24%) in last 10 min
951,2022-01-01 00:07:23,76.0,CABLE_ERROR_STATE,SUB_HIGH_CPU_USAGE,OK,Average load (24%) in last 10 min
1013,2022-01-01 00:07:55,73.0,CABLE_ERROR_STATE,LG_TEMPERATURE,OK,"CPU, 32℃"
...,...,...,...,...,...,...
3442177,2022-01-31 23:41:49,NaN,NaN,SUB_HIGH_CPU_USAGE,OK,Average load (22%) in last 10 min
3442178,2022-01-31 23:47:48,NaN,NaN,LG_TEMPERATURE,OK,"CPU, 33℃"
3442179,2022-01-31 23:51:52,NaN,NaN,SUB_HIGH_CPU_USAGE,OK,Average load (21%) in last 10 min
3442180,2022-01-31 23:56:53,NaN,NaN,SUB_HIGH_CPU_USAGE,OK,Average load (20%) in last 10 min


In [178]:
critical_13 = merge_meter_state_13[merge_meter_state_13.StationUnitState == 'CRITICAL']
critical_13.full_info.value_counts()

AttributeError: 'DataFrame' object has no attribute 'full_info'

In [126]:
critical_13.Comment.value_counts()

CableTemperatureCableTemperature (8208)     81145
CableTemperature (8208)                     51459
HasNoAnswerFromBoardHasNoAnswerFromBoard     3911
HasNoAnswerFromBoard                         1055
0% (0/5) online                                61
0% (0/0) online                                14
EmergencyButton                                12
Can bus is dead                                 5
Resource unavailable                            4
0% (0/2) online                                 1
GapCpLine                                       1
Name: Comment, dtype: int64

In [125]:
critical_13.StationUnitType.value_counts()

LG_PU_OUT           132725
LG_PU_CONV            4520
LG_E_METER_OUT         297
LG_CPU                  74
LG_CHILLER              43
SUB_CAN_BUS              5
LG_NETWORK_MODEM         4
Name: StationUnitType, dtype: int64

In [115]:
pivot_meter_state_13 = critical_13[critical_13.StationUnitState == 'CRITICAL'].pivot_table(index='full_info', 
                        columns='StationUnitType', 
                        values='Time', 
                        aggfunc='count',
                        fill_value=0).reset_index()

In [116]:
pivot_meter_state_13.sort_values(['LG_PU_OUT'], ascending=False)

StationUnitType,full_info,LG_CHILLER,LG_CPU,LG_E_METER_OUT,LG_PU_CONV,LG_PU_OUT
4,CABLE_ERROR_STATE,0,1,13,280,115281
7,CABLE_TEMPERATURE,1,1,4,81,6390
3,CABLE_CURRENT,0,3,12,120,1615
6,CABLE_POWER,0,3,12,120,1615
9,CABLE_VOLTAGE,0,3,12,120,1615
58,Y_MODEM_PING,1,2,4,44,976
57,Y_MODEM_MOBILE_SIGNAL_TYPE,1,2,4,44,960
54,Y_LOGIC_BOARD_TEMPERATURE,1,2,4,44,550
10,CABLE_VOLTAGE_AFTER_CONTACTOR,0,0,4,84,326
32,Y_E_METER_ENERGY_REGISTER,0,0,4,76,236


In [119]:
pivot_ok_13 = ok_13.pivot_table(index='full_info', 
                        columns='StationUnitType', 
                        values='Time', 
                        aggfunc='count',
                        fill_value=0).reset_index()

In [121]:
pivot_ok_13

StationUnitType,full_info,APP_LOCAL_MANAGER,APP_MONITOR,APP_OCPP,APP_V2G,LG_CHILLER,LG_CPU,LG_E_METER_OUT,LG_NETWORK_MODEM,LG_PLC_MODEM,LG_PU_CONV,LG_PU_OUT,LG_TEMPERATURE,SUB_CAN_BUS,SUB_HIGH_CPU_USAGE,SUB_HIGH_FS_USAGE,SUB_HIGH_MEMORY_USAGE,SUB_POSTGRES,SUB_VPN
0,CABLE_CONNECTOR_LOCKED,134,134,134,134,86,136,174,86,87,450,110,136,134,134,240,133,82,134
1,CABLE_CONTACTOR_LOCKED,246,246,246,246,161,248,322,161,162,824,196,248,246,246,436,244,142,246
2,CABLE_CP_LINE_STATE,134,134,134,134,87,135,172,86,87,446,108,135,134,135,240,133,82,134
3,CABLE_CURRENT,295,290,290,290,203,290,376,183,183,994,243,774,290,833,524,317,186,290
4,CABLE_ERROR_STATE,730,727,723,721,508,725,993,502,494,2485,546,3477,725,9917,1218,798,356,726
5,CABLE_ISOLATION_STATE,268,268,268,268,172,272,348,172,174,900,219,272,268,268,480,266,164,268
6,CABLE_POWER,295,290,290,290,203,290,376,183,183,994,243,774,290,833,524,317,186,290
7,CABLE_TEMPERATURE,179,179,177,179,114,181,234,115,115,631,147,429,182,952,320,190,115,181
8,CABLE_VEHICLE_CONNECT,268,268,268,268,172,270,344,172,173,891,217,271,268,269,480,266,164,268
9,CABLE_VOLTAGE,295,290,290,290,203,290,376,183,183,994,243,774,290,833,524,317,186,290


In [123]:
ok_13.StationUnitType.value_counts()

LG_PU_CONV               45716
SUB_HIGH_CPU_USAGE       29111
SUB_HIGH_FS_USAGE        24336
LG_TEMPERATURE           19580
LG_E_METER_OUT           17471
SUB_HIGH_MEMORY_USAGE    13705
LG_CPU                   13671
APP_LOCAL_MANAGER        13591
APP_MONITOR              13566
APP_OCPP                 13564
APP_V2G                  13563
SUB_CAN_BUS              13523
SUB_VPN                  13514
LG_PU_OUT                11005
LG_PLC_MODEM              8775
LG_CHILLER                8770
LG_NETWORK_MODEM          8725
SUB_POSTGRES              8436
Name: StationUnitType, dtype: int64

In [2]:
new_meter_values = pd.read_csv('meter_values_7.csv', index_col='Unnamed: 0')
new_station_state = pd.read_csv('new_station_state.csv', index_col='Unnamed: 0')

In [3]:
new_meter_values = new_meter_values.sort_values(['StationId'], ascending=True)

In [35]:
new_meter_values.to_csv('new_meter_values.csv')

In [31]:
new_meter_values.MeterUnit.unique()

array([nan])

In [33]:
new_meter_values = new_meter_values.astype({'StationId': np.int8, 'UnitId': np.int8}).drop(['StationTransactionId', 'MeterUnit'],axis=1)

In [5]:
meter_values_count = new_meter_values.pivot_table(index=['StationId','UnitId'], 
                        columns='LocationType', 
                        values='Value', 
                        aggfunc='count',
                        fill_value=0).reset_index()

In [6]:
meter_values_count

LocationType,StationId,UnitId,CABLE,EV,Y_CHILLER,Y_CONVERTER_MODULE,Y_CPU,Y_E_METER,Y_E_METER_INPUT,Y_HLC,Y_LOGIC_BOARD,Y_MODEM
0,1,0,0,0,0,62,2406,0,0,0,5418,19141
1,1,1,54857,216,597,54,410,270,0,756,216,0
2,1,2,489,0,0,54,410,270,0,378,54,0
3,1,3,0,0,0,54,0,0,0,0,0,0
4,1,4,0,0,0,54,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
187,38,35,0,0,0,56,0,0,0,0,0,0
188,38,48,0,0,0,56,0,0,0,0,0,0
189,38,49,0,0,0,56,0,0,0,0,0,0
190,38,50,0,0,0,56,0,0,0,0,0,0


In [16]:
new_station_state = new_station_state.query('StationId == [1,2,3,4,5,6,7,8,9,10,11,12,13,15,16,17,37,38]').drop(['TimestampEventRegistered'], axis=1)

In [17]:
new_station_state.tail()

,StationId,Time,UnitId,StationUnitType,StationUnitState,BoardErrorCode,StationTransactionId,Comment
2461948,38,2022-01-07 23:57:46.167 +0000,1,LG_CAMERA,OK,EmptyValue,0,NaN
2461949,38,2022-01-07 23:58:29.694 +0000,1,LG_TEMPERATURE,OK,EmptyValue,0,"CPU, 37℃"
2461950,38,2022-01-07 23:58:29.700 +0000,2,LG_TEMPERATURE,OK,EmptyValue,0,"CPU, 37℃"
2461951,38,2022-01-07 23:58:35.692 +0000,1,LG_TEMPERATURE,OK,EmptyValue,0,"CPU, 17℃"
2461952,38,2022-01-07 23:58:35.694 +0000,2,LG_TEMPERATURE,OK,EmptyValue,0,"CPU, 17℃"


In [37]:
station_state_all_count = station_state.pivot_table(index='StationId', 
                        columns='StationUnitState', 
                        values='Time', 
                        aggfunc='count',
                        fill_value=0).reset_index()

In [38]:
station_state_all_count.sort_values(['CRITICAL'], ascending=False)

StationUnitState,StationId,CRITICAL,ERROR,INFO,OK,WARNING
12,13,60315,175808,88,18305,137
1,2,21715,25058,748,17615,0
46,52,21310,1002,314,236785,356
13,14,5568,63,1,29851,177
41,47,4425,505,0,159776,0
...,...,...,...,...,...,...
10,11,211,8382,1,28104,4546
16,17,182,724,3,23410,835
4,5,28,61,1,24075,2
5,6,20,22,51,27596,0


In [67]:
station_state_count = new_station_state.pivot_table(index='StationId', 
                        columns='StationUnitState', 
                        values='BoardErrorCode', 
                        aggfunc='count',
                        fill_value=0).reset_index()

In [68]:
station_state_count

StationUnitState,StationId,CRITICAL,ERROR,INFO,OK,WARNING
0,1,201,183,0,5595,6
1,2,2240,5296,540,5872,0
2,3,82,246,0,4001,82
3,4,97,47,0,6357,92
4,5,2,5,0,5084,0
5,6,3,10,49,6493,0
6,7,283,197,0,5073,83
7,8,85,11,1,4211,84
8,9,87,5,0,5549,84
9,10,11,8,0,5267,0


In [53]:
new_station_state[new_station_state['StationUnitState']== 'OK'].StationUnitType.unique()

array(['SUB_HIGH_CPU_USAGE', 'LG_PU_CONV', 'LG_CPU', 'LG_PU_OUT',
       'LG_CHILLER', 'LG_E_METER_OUT', 'LG_NETWORK_MODEM', 'LG_PLC_MODEM',
       'LG_TEMPERATURE', 'APP_V2G', 'APP_MONITOR', 'APP_LOCAL_MANAGER',
       'SUB_CAN_BUS', 'SUB_HIGH_FS_USAGE', 'SUB_HIGH_MEMORY_USAGE',
       'SUB_VPN', 'APP_OCPP', 'LG_E_METER_IN', 'SUB_POSTGRES',
       'LG_CAMERA'], dtype=object)

In [54]:
new_station_state.shape

(284025, 9)

In [55]:
#OK
new_station_state[new_station_state['StationUnitState']== 'OK'].shape

(169734, 9)

In [56]:
#INFO
new_station_state[new_station_state['StationUnitState']== 'INFO'].shape

(660, 9)

In [57]:
#ERROR
new_station_state[new_station_state['StationUnitState']== 'ERROR'].shape

(82459, 9)

In [58]:
#CRITICAL
new_station_state[new_station_state['StationUnitState']== 'CRITICAL'].shape

(30304, 9)

In [59]:
#WARNING
new_station_state[new_station_state['StationUnitState']== 'WARNING'].shape

(868, 9)

In [60]:
new_station_state.StationUnitState.unique()

array(['OK', 'CRITICAL', 'ERROR', 'WARNING', 'INFO'], dtype=object)

In [ ]:
# df['CABLE_CONNECTOR_LOCKED'].astype(int)
# df['CABLE_CONTACTOR_LOCKED'].astype(float) 
# df['CABLE_CP_LINE_STATE'].astype(int)
# df['CABLE_CURRENT'].astype(float)
# df['CABLE_ERROR_STATE'].astype(float)
# df['CABLE_ISOLATION_STATE'].astype(float)
# df['CABLE_POWER'].astype(float)
# df['CABLE_TEMPERATURE'].astype(float)
# df['CABLE_VEHICLE_CONNECT'].astype(float)
# df['CABLE_VOLTAGE'].astype(float)
# df['CABLE_VOLTAGE_AFTER_CONTACTOR'].astype(float)
# df['EV_EVCC_ID'].astype(int)
# df['EV_TARGET_CHARGING_ALLOW'].astype(float)
# df['EV_TARGET_CURRENT'].astype(int)
# df['EV_TARGET_VOLTAGE'].astype(float)
# df['Y_CHILLER_CHILLER_STATE'].astype(float)
# df['Y_CHILLER_CHILLER_WATER_RUN'].astype(int)
# df['Y_CHILLER_ERROR_STATE'].astype(int)
# df['Y_CHILLER_TEMPERATURE'].astype(int)
# df['Y_CONVERTER_MODULE_ERROR_STATE'].astype(float)
# df['Y_CPU_BOARD_READY'].astype(float)
# df['Y_CPU_CURRENT'].astype(float)
# df['Y_CPU_CURRENT_MAX'].astype(int)
# df['Y_CPU_ERROR_STATE'].astype(int)
# df['Y_CPU_FAN_ACTIVE'].astype(int)
# df['Y_CPU_POWER'].astype(float)
# df['Y_CPU_POWER_MAX'].astype(int)
# df['Y_CPU_TEMPERATURE'].astype(float)
# df['Y_CPU_VOLTAGE'].astype(float)
# df['Y_CPU_VOLTAGE_INPUT'].astype(int)
# df['Y_CPU_VOLTAGE_MAX'].astype(int)
# df['Y_E_METER_CURRENT'].astype(float)
# df['Y_E_METER_ENERGY_REGISTER'].astype(int)
# df['Y_E_METER_ERROR_STATE'].astype(int)
# df['Y_E_METER_POWER'].astype(float)
# df['Y_E_METER_VOLTAGE'].astype(int)
# df['Y_HLC_AUTHENTICATION'].astype(int)
# df['Y_HLC_BATTERY_LEVEL'].astype(float)
# df['Y_HLC_BOARD_READY'].astype(int)
# df['Y_HLC_CHARGING_ALLOW'].astype(float)
# df['Y_HLC_CHARGING_GOING'].astype(float)
# df['Y_HLC_EV_ERROR_STATE'].astype(float)
# df['Y_HLC_HLC_STATE'].astype(float)
# df['Y_HLC_SLAC_AAG'].astype(float)
# df['Y_HLC_TARGET_CHARGING_ALLOW'].astype(float)
# df['Y_HLC_TARGET_CURRENT'].astype(float)
# df['Y_HLC_TARGET_CURRENT_LIMIT'].astype(int)
# df['Y_HLC_TARGET_POWER_LIMIT'].astype(int)
# df['Y_HLC_TARGET_VOLTAGE'].astype(float)
# df['Y_HLC_TARGET_VOLTAGE_LIMIT'].astype(int)
# df['Y_LOGIC_BOARD_CHILLER_WATER_RUN'].astype(int)
# df['Y_LOGIC_BOARD_CURRENT_MAX'].astype(float)
# df['Y_LOGIC_BOARD_ENERGY_ACTIVE_IMPORT_REGISTER_FROM_0'].astype(int)
# df['Y_MODEM_PING'].astype(int)

In [35]:
merge_meter_state_13 = pd.read_csv('merge_meter_state_13.csv', index_col='Unnamed: 0').drop('Time', axis=1)

In [36]:
merge_meter_state_13

,y,CABLE_CONNECTOR_LOCKED,CABLE_CONTACTOR_LOCKED,CABLE_CP_LINE_STATE,CABLE_CURRENT,CABLE_ERROR_STATE,CABLE_ISOLATION_STATE,CABLE_POWER,CABLE_TEMPERATURE,CABLE_VEHICLE_CONNECT,...,Y_HLC_TARGET_VOLTAGE_LIMIT,Y_LOGIC_BOARD_CHILLER_WATER_RUN,Y_LOGIC_BOARD_CURRENT_MAX,Y_LOGIC_BOARD_ENERGY_ACTIVE_IMPORT_REGISTER_FROM_0,Y_LOGIC_BOARD_POWER_MAX,Y_LOGIC_BOARD_TEMPERATURE,Y_LOGIC_BOARD_VOLTAGE_MAX,Y_MODEM_MOBILE_SIGNAL_STRENGTH,Y_MODEM_MOBILE_SIGNAL_TYPE,Y_MODEM_PING
0,1,0.0,0.0,0.0,0.0,74.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,74.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.0,0.0,0.0,0.0,74.5,0.0,0.0,42.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0.0,0.0,0.0,0.0,74.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0.0,0.0,0.0,0.0,74.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78608,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78610,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78611,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78613,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
X = merge_meter_state_13.drop('y', axis=1)
y = merge_meter_state_13.y

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1234)

In [44]:
X_train['y'] = y_train

In [ ]:
X_train.to_csv('train.csv')

In [43]:
X_test.to_csv('test.csv')